# Python am EBC

In diesem Notebook sind Beispiel zu den Themen dargestellt, die im Workshop angesprochen worden sind.


## Verwendung von Objektorientierung zur effizienten Auswertung
Mittels Objektorientierung ist es häufig leicht möglich, Programmcode mehrfach zu verwenden. Insbesondere bei vergleichenden Untersuchungen, bei denen immer wieder ähnliche Daten anfallen ist dies von großem Vorfall. Objektorientierung allerdings eine etwas gründlicher Vorplanung. 

Im folgenden wird eine Basisklasse für Simulations oder Messdaten erstellt:

In [ ]:
%matplotlib notebook
import os.path
import pandas as pd
import pylab as plt

class BaseData(object):
    ''' Keep simulation or measurement data
    
        Parameters
        -----------
        filename : str
            path to filename
            
        name : str, optional [None]
            It is always a good idea to label your data, if None, name will
            be determined from filename
        
        Attributes
        -----------
        data : pd.Series
            data read from result file, c
            
        savedir : str
            path were results/figures etc. should be saved, defaults to 'D:/'
    '''
    
    def __init__(self, filename, name=None):
        self.filename = filename
        self.savedir = 'D:/'
        self.data = self.read_data()
        if name is None:
            _dummy = (os.path.basename(filename).split('.')[:-1]) # Get the filename without suffix
            self.name = ''.join(_dummy) # Rebuild a string from resulting list
        else:
            self.name = name
            
        
    def read_data(self):
        ''' Read data from input file
        
            Returns:
            ----------
            pd.DataFrame
        '''
        data = pd.Series.from_csv(self.filename, header=0)
        return data
    
    def plot_data(self, label=None, fig=None, ax=None):
        ''' Plot the data into a figure
        
            Parameters:
            ----------
            label : str, optional [self.name]
                Identifier for this function (used in the legend)
            fig : matplotlib.figure instance, optional [None]
                Figure to plot into, will be created if None
            ax : matplotlib.axes instance, optional [None]
                Axes to plot into, will be created if None
        
            Returns:
            ----------
            None
            
            Notes:
            ----------
            In the EBC-Python Library exist an advanced helper to create figures and axes
            (the first few lines in this code), see base_functions.reusable_helpers.helper_figures
        
        '''
        if fig is None and ax is None:
            fig, ax = plt.subplots()
        elif fig is None:
            fig = ax.get_figure()
        elif ax is None:
            ax = fig.add_axes()
        if label is None:
            label = self.name
        self.data.plot(ax=ax, label=label)
        ax.set_title('Leistung über Ventilstellung' )
        ax.set_ylabel('Leistung in kW')
        ax.legend()
        
a = BaseData('Referenz.csv')
b = BaseData('Messung1.csv')
a.plot_data('Referenz')




Die Daten der beiden lassen sich jetzt aber auch problemlos zusammen darstellen

In [ ]:
fig, ax = plt.subplots() # Leere Figure mit Achsen erstellen
a.plot_data('Referenz',fig=fig, ax=ax)# Nutzen der optionalen Parameter
b.plot_data(fig=fig, ax=ax) # Label muss nicht angegeben werden

Mit dem Programmcode werden beim Ablauf sowohl ein Referenzwert als auch eine erste Messung eingelesen. Für weitere Messungen wurde auf das überlegene Programm "Excel" gewechselt, damit kann die ursprüngliche Klasse die Daten leider nicht einlesen, alle anderen Funktionen sollten aber noch erhalten bleiben. Insofern wird ein neues Objekt von BaseData abgeleitet und die Einleseprozedur angepasst. Alle anderen Funktionen werden von BaseData geeerbt, einschließlich der \__init__ Prozedur

In [ ]:
class NewData(BaseData):
    ''' Modified Version of BaseData, only reading Method is changed
    '''
          
    def read_data(self):
        ''' Read data from input file
        
            Returns:
            ----------
            pd.DataFrame
        '''
        data = pd.read_excel(self.filename, index_col = 0)
        data=data.iloc[:,0] # Transform pd.DataFrame to pd.Series
        return data
c = NewData('Messung2.xlsx')


Die Daten können jetzt in eine Klasse überführt werden, die eine Auswertung ermöglichen

In [ ]:
class Comparison(object):
    def __init__(self):
        self.members = [] # Keep a list of the added results (in contrast to the dictionary below, this will keep the order the data were added)
        self.members_by_name = {} # If you would like to access them by name directly
        self.reference = None
        
    def add_result(self, obj):
        ''' Add results of one the measurement the attribute `members`.

        Parameters:
        ----------
        obj : Instance of BaseData (or inherited)

        Returns:
        ----------
        None

        '''
        assert isinstance(obj, BaseData), 'Obj must be an instance of BaseData, but got {0}'.format(type(obj))
        assert obj.name not in list(self.members_by_name.keys()), 'Measurement with name {0} already exists'.format(obj.name)
        self.members.append(obj)
        self.members_by_name[obj.name] = obj
        
    def set_reference(self, obj):
        assert isinstance(obj, BaseData), 'Obj must be an instance of BaseData, but got {0}'.format(type(obj))
        self.reference = obj
        
        
    def plot_relative(self, result, fig, ax):
        ''' Plot the measured data relative to the reference data
        
            Parameters:
            ----------
            result : BaseData object
                The result that should be plotted
                
            fig : matplotlib.figure instance
                Figure to plot into, will be created if None
                
            ax : matplotlib.axes instance
                Axes to plot into, will be created if None
        '''
        rel = result.data / self.reference.data
        rel.plot(ax=ax, label = result.name)
        ax.set_ylabel('Relative Leistung zur Referenz')
        ax.legend()

# Daten zur Klasse hinzufügen
all = Comparison()
all.add_result(a)
# all.add_result(a) # Adding this a second time is prevented by the assert statement
all.add_result(b)
all.add_result(c)
all.set_reference(all.members_by_name['Referenz'])
print(all.members)

# Alle Ergebnisse in einen Plot
fig, ax = plt.subplots()
for member in all.members:
    all.plot_relative(member, fig, ax)
    
# Alternativ als Plot nebeneinander
fig, ax = plt.subplots(2, sharex=True)
for i, member in enumerate(all.members[1:]):
    all.plot_relative(member, fig, ax[i])
ax[0].set_xlabel('') # Doppeltes Label an der X-Achse entfernen